![Podmanlogo](Pictures/podman-logo.png)

# Podman hello world: pulling and executing your first container image

Just like any other container engine, Podman has the ability to manage and execute container images. A container image is a static image that contains an applications with all it's dependencies. Because the dependencies are included within the container image we can expect the same application behaviour if it's executed in different hosts, solving one of the most widespread problems for developers: "it worked in my environment".

Container images are stored in container registries, by default Podman will look for container images in quay.io, registry.redhat.io and docker.io. You can configure additional registries by modifying the file /etc/containers/registries.conf, we'll not cover this advanced configuration during this workshop.

The way Podman works with container images is it locally downloads (or pulls) images from a registry. Hence, first thing we want to do is checking if we have any image downloaded locally:

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Podman101'] }}

In [ ]:
podman images

You should have no images locally. Download your first image, the podman hello-world image, by running the following command:

In [ ]:
podman pull hello-world

Now check if your image was pulled correctly by running again the podman images command:

In [ ]:
podman images

You should find the hello-world image is locally downloaded in your system. This only means that we have a static image that contains our application and its dependancies locally sync'ed. Next thing we would like to do is to execute that static image:

In [ ]:
podman run hello-world

As you can see, podman needs to pull (download) an image in order to execute it. Also, if you want to get detailed information about the image with podman you would need to pull it as podman will only work with locally available images. If you wanted to get information or inspect a remote image without downloading it you can use one of the tools in podman ecosystem: skopeo.

Lets try inspecting a remote container image residing in docker.io without pulling it, therefore with skopeo.

In [ ]:
skopeo inspect docker://docker.io/redhat/ubi9-minimal

Pay attention to the naming convention, you need to specify "docker://" before the repo, container name and tag.

As you can see we are able to get the information about the container image but we did not need to pull the image for that. Lets check that the image was not downloaded.

In [ ]:
podman images

Another very intereting and widely used skopeo functionality is copying images from a source registry to a target registry with the "skopeo copy" command, by using it you don't need to have a copy of the image in your local system.

# Executing containerized applications

During last exercise we downloaded a container image and executed it. With the following command Podman will show the containers that are being executed right now:

In [ ]:
podman ps

As you can see, the output is empty. This is because the container that we executed runs a process that ends after printing in the screen the "Hello Podman World" message and once the main process of a container is finished the container itself will stop being executed.
But no worries, Podman got us covered, if we want to see all the containers that are being executed and those whose execution finished some time ago we just need to add the "-a" to the previous command.

In [ ]:
podman ps -a

There you can see the container we executed before with a status of "Exited".

Lets see what happens if we execute a container whose process don't finish unless specified:

In [ ]:
podman run nginx

You need to stop the process manually as the container process has taken all of your terminal. To do that you need to press the stop button on top of the jupyter notebook, as shown in this picture:

![stopping-nginx](Pictures/stopping-nginx.png)

If you want to run a container in the background, then you should add the option --detach (or simply -d).

In [ ]:
podman run --detach nginx

The container is now being executed but it's not taking over the terminal. Now, if you run the "podman ps" command you will see your nginx container with a status of "Up".

In [ ]:
podman ps

You see the container has an ID and also a name. In this case the name was randomly generated as we did not specify any name, pay attention to next time we execute a container image as we will be giving it a name.

You can stop all running containers by running the following:

In [ ]:
podman stop --all

Now that all containers have beeen stopped run again "podman ps" to check if all of them were correctly stopped.

In [ ]:
podman ps

As you can see the container is not present because "podman ps" only shows the containers that are being executed right now. If we use the "-a" option it will show all containers that are in execution right now but also those that were stopped, it'll show the stopped ones with an status of "Exited".

In [ ]:
podman ps -a

This means that containers are not deleted once the process is finished and you could simply restart any of them. We can do this by using the container ID, but for the sake of simplicity we will be creating a new container with a defined name:

In [ ]:
podman run -d --name my-podman-container nginx
podman ps

See how we changed the long "--detach" option for a short "-d" but it works the same way. We also defined the name. So now we can just stop it by it's name.

In [ ]:
podman stop my-podman-container
podman ps -a

Now that it is stopped, we can restart it.

In [ ]:
podman start my-podman-container
podman ps

While the container is running you can also inspect it to get to know all the information about it.

In [ ]:
podman inspect my-podman-container

If we stop it again we see that we can also remove it by running the command "podman rm"

In [ ]:
podman stop my-podman-container
podman rm my-podman-container

Now it has been fully removed so we wont be able to start that same container again, we will need to create a new container from the same image. You can check it is fully removed by looking at the empty output of "podman ps -a"

In [ ]:
podman ps -a

# Interacting with containers

A very common question is how to sneek into a running container and anaylse it's file content. There are several ways you can do this, one of the most common is to run the container interactively. You will need to add two options for this purpose, first one is the "--tty" that will allocate a pseudo-TTY for the container; the second is "--interactive" which will keep STDIN open even if not attached. We can combine both options just by using the "-it" option.
Then you will need to specify the command you want to run in your container. If you specify "/bin/bash" as the command you want to run you will open an interactive terminal with your container.

In our case, this environment based on jupyter notebooks doesn't work well with interactive commands. We will need to follow a special process to test this.

First open a new tab by clicking on the "+" sign on top of the notebook.

![interactive-container-1](Pictures/interactive-container-1.png)

In the launcher tab select the terminal option.

![interactive-container-2](Pictures/interactive-container-2.png)

Then ssh into the remote podman machine 16.31.86.187. Remember to use your student number which may be different from the one in the picture. Also, if you get asked, accept the connection to the host by typing "yes".

![interactive-container-3](Pictures/interactive-container-3.png)

Now you can run the command "podman run -it --rm --name my-interactive-container fedora /bin/bash" and you will have access to an interactive terminal within the container. Test it by running some commands like "whoami", "pwd", "ls /" or "cat /etc/fedora-release".

![interactive-container-4](Pictures/interactive-container-4.png)

Once you have finished type "exit", close the terminal tab and continue with the workshop.

Sometimes, instead of opening an interactive terminal you just want to execute a single command within a container and see the output. You can do that in a very simple way, lets first deploy a container.


In [ ]:
podman run -d --name my-podman-container fedora sleep 300

Once the container is running you can just use the "podman exec" command and then specify the command that you want to run.

In [ ]:
podman exec my-podman-container cat /etc/fedora-release

As you can see the output showed the version of Fedora running within the container.

# Cleanup

In [ ]:
podman stop --all
podman rm --all
podman rmi hello-world nginx fedora

In [ ]:
%logout

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#631f61;"></i>&nbsp;&nbsp;Next Steps

# Lab 3 : Managing container images

<h2>Next LAB&nbsp;&nbsp;&nbsp;&nbsp;<a href="3-WKSHP-Managing-container-images.ipynb" target="New" title="Next LAB: Managing container images"><i class="fas fa-chevron-circle-right" style="color:#631f61;"></i></a></h2>

</br>
 <a href="2-WKSHP-Podman-hello-world.ipynb" target="New" title="Back: Podman hello-world"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#631f61;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="3-WKSHP-Managing-container-images.ipynb" target="New" title="Next:Managing container images"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#631f61;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
